## H2O Model Playground

In [4]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

file_path = "./data/lg_all_features.csv"

In [2]:
h2o.cluster().shutdown()

AttributeError: 'NoneType' object has no attribute 'shutdown'

In [3]:
from pyspark.sql import SparkSession
from pysparkling import *
import h2o



# Create Spark session
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

h2o.init()
# Initialize H2OContext
hc = H2OContext.getOrCreate()

# Get H2OConf from H2OContext
h2o_conf = hc.getConf()

# Set H2OConf properties
h2o_conf.set("spark.ext.h2o.client.language", "python")


24/01/21 19:34:33 WARN Utils: Your hostname, Jacobs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.4.156 instead (on interface en0)
24/01/21 19:34:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/01/21 19:34:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/21 19:34:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/01/21 19:34:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 53 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,sparkling-water-jacobfletcher_local-1705869136820
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,821 Mb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


24/01/21 19:34:35 WARN InternalH2OBackend: Increasing 'spark.locality.wait' to value 0 (Infinitive) as we need to ensure we run on the nodes with H2O
24/01/21 19:34:35 WARN NativeLibrary: Cannot load library from path lib/osx_64/libxgboost4j_minimal.dylib
24/01/21 19:34:35 WARN NativeLibrary: Failed to load library from both native path and jar!
24/01/21 19:34:35 WARN XGBoostExtension: Cannot initialize XGBoost backend! Xgboost (enabled GPUs) needs: 
  - CUDA 8.0
XGboost (minimal version) needs: 
  - GCC 4.7+
For more details, run in debug mode: `java -Dlog4j.configuration=file:///tmp/log4j.properties -jar h2o.jar`



01-21 19:34:36.314 192.168.4.156:54325   30904    Thread-3  INFO water.default: ----- H2O started  -----
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Build git branch: rel-3.44.0
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Build git hash: 3f9977a3972e857f0ffb7b1925c75f972f9a07b1
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Build git describe: jenkins-3.44.0.2-25-g3f9977a
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Build project version: 3.44.0.3
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Build age: 1 month and 1 day
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Built by: 'jenkins'
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Built on: '2023-12-20 16:31:48'
01-21 19:34:36.315 192.168.4.156:54325   30904    Thread-3  INFO water.default: Found H

H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,sparkling-water-jacobfletcher_local-1705883674494
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.44.0.3-1-3.5
 * H2O name: sparkling-water-jacobfletcher_local-1705883674494
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,192.168.4.156,54325)
  ------------------------

  Open H2O Flow in browser: http://192.168.4.156:54324 (CMD + click in Mac OSX)

    
Sparkling Water configuration:
  backend cluster mode : internal
  workers              : None
  cloudName            : sparkling-water-jacobfletcher_local-1705883674494
  base port            : 54321
  cloudTimeout         : 60000
  log level            : INFO
  nthreads             : -1
  drddMulFactor        : 10


In [5]:
h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"




01-21 19:35:04.325 192.168.4.156:54325   30904  5100564-94  INFO water.default: POST /3/ImportFilesMulti, parms: {paths=[/Users/jacobfletcher/git/churn_project/data/lg_all_features.csv]}
01-21 19:35:04.382 192.168.4.156:54325   30904  5100564-80  INFO water.default: POST /3/ParseSetup, parms: {source_frames=["nfs://Users/jacobfletcher/git/churn_project/data/lg_all_features.csv"], single_quotes=False, check_header=0}
01-21 19:35:04.695 192.168.4.156:54325   30904  5100564-80  INFO water.default: ParseSetup heuristic: cloudSize: 1, cores: 12, numCols: 43, maxLineLength: 177, totalSize: 3462322, localParseSize: 3462322, chunkSize: 72132, numChunks: 47, numChunks * cols: 2021
01-21 19:35:04.706 192.168.4.156:54325   30904  5100564-94  INFO water.default: POST /3/Parse, parms: {chunk_size=72132, single_quotes=False, delete_on_done=True, parse_type=CSV, check_header=1, separator=44, escapechar=0, column_types=["Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","N

In [6]:


# Define hyperparameter grid
hyperparams = {
    'alpha': [0.0, 0.2, 0.4, 0.6, 0.8, 1.0],  # Regularization parameter
    'lambda': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0],
}

# Initialize the linear regression model
linear_reg = H2OGeneralizedLinearEstimator(family='binomial', link="logit")

# Perform grid search
grid_search = H2OGridSearch(linear_reg, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

# Get the best model from the grid search
best_model = grid_search.get_grid()[0]

# Make predictions on the validation set
preds = best_model.predict(valid)

# Get model performance on the validation set
validation_performance = best_model.model_performance(valid)


01-21 19:35:16.139 192.168.4.156:54325   30904  5100564-80  INFO water.default: POST /99/Grid/glm, parms: {fold_assignment=auto, calc_like=False, build_null_model=False, theta=1e-10, generate_scoring_history=False, compute_p_values=False, fix_dispersion_parameter=False, score_each_iteration=False, gradient_epsilon=-1.0, prior=-1.0, max_active_predictors=-1, remove_collinear_columns=False, hyper_parameters={"alpha": [0.0,0.2,0.4,0.6,0.8,1.0],"lambda": [1e-05,0.0001,0.001,0.01,0.1,1.0]}, solver=auto, max_iterations=-1, keep_cross_validation_predictions=False, tweedie_epsilon=8e-17, auc_type=auto, stopping_metric=auto, lambda_search=False, training_frame=py_3_sid_931a, stopping_tolerance=0.001, init_dispersion_parameter=1.0, score_iteration_interval=-1, tweedie_variance_power=0.0, intercept=True, max_confusion_matrix_size=20, dispersion_learning_rate=0.5, fix_tweedie_variance_power=True, non_negative=False, stopping_rounds=0, seed=-1, missing_values_handling=mean_imputation, dispersion_ep

Adding alpha array to hyperparameter runs slower with gridsearch. This is due to the fact that the algo has to run initialization for every alpha value. Setting the alpha array as a model parameter will skip the initialization and run faster overall.


01-21 19:35:18.885 192.168.4.156:54325   30904  5100564-75  INFO water.default: GET /3/Models/Grid_GLM_py_3_sid_931a_model_python_1705883675495_1_model_3, parms: {}
01-21 19:35:18.895 192.168.4.156:54325   30904  5100564-94  INFO water.default: GET /3/Models/Grid_GLM_py_3_sid_931a_model_python_1705883675495_1_model_4, parms: {}
01-21 19:35:18.906 192.168.4.156:54325   30904  5100564-75  INFO water.default: GET /3/Models/Grid_GLM_py_3_sid_931a_model_python_1705883675495_1_model_5, parms: {}
01-21 19:35:18.916 192.168.4.156:54325   30904  5100564-94  INFO water.default: GET /3/Models/Grid_GLM_py_3_sid_931a_model_python_1705883675495_1_model_6, parms: {}
01-21 19:35:18.925 192.168.4.156:54325   30904  5100564-75  INFO water.default: GET /3/Models/Grid_GLM_py_3_sid_931a_model_python_1705883675495_1_model_7, parms: {}
01-21 19:35:18.934 192.168.4.156:54325   30904  5100564-94  INFO water.default: GET /3/Models/Grid_GLM_py_3_sid_931a_model_python_1705883675495_1_model_9, parms: {}
01-21 19:3

In [7]:

print(validation_performance.F1())
print(validation_performance.F2())

[[0.29435596636523476, 0.5339805825242718]]
[[0.1601108553305616, 0.6394192521877485]]


In [8]:
validation_performance

ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.14407239832791754
RMSE: 0.3795687004060234
LogLoss: 0.4600886755208726
AUC: 0.7642154006391385
AUCPR: 0.5374468508647073
Gini: 0.528430801278277
Null degrees of freedom: 3337
Residual degrees of freedom: 3301
Null deviance: 3596.572639562097
Residual deviance: 3071.551997777346
AIC: 3145.551997777346

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.29435596636523476
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      2130  442  0.1719   (442.0/2572.0)
1      326   440  0.4256   (326.0/766.0)
Total  2456  882  0.2301   (768.0/3338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.294356     0.533981  187
max f2                       0.160111     0.639419  281
max f0point5                 0.412868     0.581348  124
max accuracy                 0.423851     0.815159  119
max precision                0.968467     1         0
max recall                   0.00294873   1         398
max specificity              0.968467     1         0
max absolute_mcc             0.368829     0.408652  146
max min_per_class_accuracy   0.235017     0.70235   228
max mean_per_class_accuracy  0.243068     0.705401  222
max tns                      0.968467     2572      0
max fns                      0.968467     765       0
max fps                      0.00100958   2572      399
max tps                      0.00294873   766       398
max tnr                      0.968467     1         0
max fnr                      0.968467     0.998695  0
max fpr                      0.00100958   1         399
max tpr                      0.00294873   1         398

Gains/Lift Table: Avg response rate: 22.95 %, avg score: 22.53 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101857                   0.74295            3.33236   3.33236            0.764706         0.827109   0.764706                    0.827109            0.0339426       0.0339426                  233.236   233.236            0.0308321
2        0.0200719                   0.691293           2.90513   3.12194            0.666667         0.712396   0.716418                    0.770608            0.0287206       0.0626632                  190.513   212.194            0.0552759
3        0.0302576                   0.649999           2.94786   3.06334            0.676471         0.672829   0.70297                     0.737693            0.0300261       0.0926893                  194.786   206.334            0.0810252
4        0.0401438                   0.608784           2.90513   3.02438            0.666667         0.629622   0.69403                     0.711078            0.0287206       0.12141                    190.513   202.438            0.105469
5        0.05003                     0.568513           3.43334   3.10519            0.787879         0.589504   0.712575                    0.687055            0.0339426       0.155352                   243.334   210.519            0.13669
6        0.10006                     0.450785           3.053     3.0791             0.700599         0.498746   0.706587                    0.5929              0.152742        0.308094                   205.3     207.91             0.269991
7        0.15009                     0.385216           2.08752   2.74857            0.479042         0.413482   0.630739                    0.533094         

## H2O Gradient Boosting

In [9]:



h2o_df = h2o.import_file(file_path)
# h2o_df = spark.read.option("header", True).csv(file_path)
h2o_df = h2o_df.drop("userId")

try:
    h2o_df.drop('C1')
    h2o_df.drop('userId')
except Exception as e:
    pass

col_features = h2o_df.columns
col_features.remove("label")
# Split the data into training and validation sets
# Split the data into training and validation sets
train, valid = h2o_df.split_frame(ratios=[0.85])

# Define predictor and response columns
predictor_cols = col_features
response_col = "label"


# Define hyperparameter grid
hyperparams = {
    'learn_rate': [0.01, 0.1, 0.2],
    'ntrees': [50, 100, 200],
    'max_depth': [3, 4, 5],
}

gradient_boost = H2OGradientBoostingEstimator(seed=42)


# Perform grid search
grid_search = H2OGridSearch(gradient_boost, hyperparams)
grid_search.train(x=predictor_cols, y=response_col, training_frame=train)

# Get the best model from the grid search
best_model = grid_search.get_grid()[0]

# Make predictions on the validation set
preds = best_model.predict(valid)

# Get model performance on the validation set
validation_performance = best_model.model_performance(valid)



01-21 19:35:30.877 192.168.4.156:54325   30904  5100564-94  INFO water.default: POST /3/ImportFilesMulti, parms: {paths=[/Users/jacobfletcher/git/churn_project/data/lg_all_features.csv]}
01-21 19:35:30.882 192.168.4.156:54325   30904  5100564-91  INFO water.default: POST /3/ParseSetup, parms: {source_frames=["nfs://Users/jacobfletcher/git/churn_project/data/lg_all_features.csv"], single_quotes=False, check_header=0}
01-21 19:35:31.132 192.168.4.156:54325   30904  5100564-91  INFO water.default: ParseSetup heuristic: cloudSize: 1, cores: 12, numCols: 43, maxLineLength: 177, totalSize: 3462322, localParseSize: 3462322, chunkSize: 72132, numChunks: 47, numChunks * cols: 2021
01-21 19:35:31.135 192.168.4.156:54325   30904  5100564-94  INFO water.default: POST /3/Parse, parms: {chunk_size=72132, single_quotes=False, delete_on_done=True, parse_type=CSV, check_header=1, separator=44, escapechar=0, column_types=["Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","N

In [ ]:

validation_performance

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.15326187011957534
RMSE: 0.39148674322328636
MAE: 0.29938115856573133
RMSLE: 0.27569242700388985
Mean Residual Deviance: 0.15326187011957534

In [11]:
h2o.cluster().shutdown()

01-21 19:37:11.611 192.168.4.156:54325   30904  5100564-91  INFO water.default: GET /, parms: {}
01-21 19:37:11.615 192.168.4.156:54325   30904  100564-116  INFO water.default: GET /flow/index.html, parms: {}
01-21 19:37:11.622 192.168.4.156:54325   30904  5100564-91  INFO water.default: POST /3/Shutdown, parms: {}
01-21 19:37:11.630 192.168.4.156:54325   30904  100564-116  INFO water.default: DELETE /4/sessions/_sid_931a, parms: {}
H2O session _sid_931a closed.


01-21 19:37:13.629 192.168.4.156:54325   30904   Thread-77  INFO water.default: Orderly shutdown:  Shutting down now.
01-21 19:37:13.633 192.168.4.156:54325   30904   Thread-77  INFO ai.h2o.org.eclipse.jetty.server.AbstractConnector: Stopped ServerConnector@47417a00{HTTP/1.1,[http/1.1]}{0.0.0.0:54325}
01-21 19:37:13.635 192.168.4.156:54325   30904   Thread-77  INFO ai.h2o.org.eclipse.jetty.server.handler.ContextHandler: Stopped a.h.o.e.j.s.ServletContextHandler@3eff00cb{/,null,UNAVAILABLE}
01-21 19:37:13.643 192.168.4.156:54325   30904  own-hook-0  INFO org.apache.spark.SparkContext: Invoking stop() from shutdown hook
01-21 19:37:13.643 192.168.4.156:54325   30904  own-hook-0  INFO org.apache.spark.SparkContext: SparkContext is stopping with exitCode 0.
01-21 19:37:13.647 192.168.4.156:54325   30904  own-hook-0  INFO org.sparkproject.jetty.server.AbstractConnector: Stopped Spark@6259ae6{HTTP/1.1, (http/1.1)}{0.0.0.0:4042}
01-21 19:37:13.650 192.168.4.156:54325   30904  own-hook-0  INFO